# Installs & Imports

In [13]:
!pip install elevenlabs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from elevenlabs import generate, play, clone, save
from elevenlabs.api import Voices
from elevenlabs.simple import set_api_key
import os
from google.colab import files
import shutil
import string
import re

# Quick Demo

In [ ]:
audio = generate(
  text='Hi! My name is Bella, nice to meet you!',
  voice= 'Bella',
  model= 'eleven_monolingual_v1' # eleven_monolingual_v1 is just for English
)

In [ ]:
play(audio, notebook = True)

In [15]:
set_api_key('be67b217717a1a7e631681d0c85395d4')

NOTE: Voice Cloning and commercial licenses are only available on paid subscriptions (\$5 for 30,000 characters (\~40 min of audio)/month, $22 for 100,000 characters (~2 hrs of audio)/month, etc.). We can create up to 3 voices with limited flexibility for free, but they would sound distinctly different from the audiobible. Other voice cloning programs aren't as convincing/realistic.

In [18]:
# Lists all voices for the authenticated user, or the default voices if no API key is set
voices = Voices.from_api()
voices

Voices(voices=[Voice(voice_id='21m00Tcm4TlvDq8ikWAM', name='Rachel', category='premade', description=None, labels={}, samples=None, settings=VoiceSettings(stability=0.0, similarity_boost=0.75), design=None), Voice(voice_id='AZnzlk1XvdvUeBnXmlld', name='Domi', category='premade', description=None, labels={}, samples=None, settings=VoiceSettings(stability=0.75, similarity_boost=0.75), design=None), Voice(voice_id='EXAVITQu4vr4xnSDxMaL', name='Bella', category='premade', description=None, labels={}, samples=None, settings=VoiceSettings(stability=0.75, similarity_boost=0.75), design=None), Voice(voice_id='ErXwobaYiN019PkySvjV', name='Antoni', category='premade', description=None, labels={}, samples=None, settings=VoiceSettings(stability=0.75, similarity_boost=0.75), design=None), Voice(voice_id='MF3mGyEYCl7XYWbV9V6O', name='Elli', category='premade', description=None, labels={}, samples=None, settings=VoiceSettings(stability=0.75, similarity_boost=0.75), design=None), Voice(voice_id='TxGEq

In [20]:
voices[-2]

Voice(voice_id='J16nzqxFhkQhOadf18wF', name='NVI_22_v2', category='cloned', description='', labels={}, samples=[VoiceSample(sample_id='1MO22xv5ZJ0yaSSNjiXG', file_name='19_Psalm 089_3.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='085a055239082dc71d41c597b68dd0bf'), VoiceSample(sample_id='4iQIUkX3sAeDP6hBIwsv', file_name='19_Psalm 018_5.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='980c7d500c8525a6267cb944ce945a69'), VoiceSample(sample_id='5SjHdln4EY57jNdEd5WN', file_name='19_Psalm 018_0.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='82d3c5894cc0d388ff3d002e70d4fc16'), VoiceSample(sample_id='6VHzNL47bkrQTMRtbkwq', file_name='19_Psalm 078_3.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='fc4a7a1a52dfef423e7a426017016853'), VoiceSample(sample_id='7UDu5Eh7ly1KXjhj6BAd', file_name='19_Psalm 018_2.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='d2043ffbc310ee62f3d3ad98093def14'), VoiceSample(sample_id='BcAzaTeCi3ocSFomkKB1', file_name='19_Psalm 11

# Compare Texts & Gather Some Statistics

Clean text for easy comparison

In [21]:
punct = string.punctuation + '«¡»'

In [22]:
def get_references(verses):
  verse_dictionary = {}

  for verse in verses:
    match = re.match(r'^(\d*\s*[A-Za-z]+\s\d+:\d+(?:-\d+)?)\t(.*)', verse)
    if match:
        verse_reference = match.group(1)
        verse_text = match.group(2).strip()

        verse_text = verse_text.translate(str.maketrans(punct, ' ' * len(punct)))
        verse_text = verse_text.lower()
        verse_text = ' '.join(verse_text.split())

        if '-' in verse_reference:
            start_verse, end_verse = verse_reference.split('-')
            chapter, start_verse_number = start_verse.split(':')
            end_verse_number = end_verse
            for verse_number in range(int(start_verse_number), int(end_verse_number) + 1):
                if verse_number == int(start_verse_number):
                    verse_dictionary[f'{chapter}:{verse_number}'] = verse_text
                else:
                    verse_dictionary[f'{chapter}:{verse_number}'] = None
        else:
            verse_dictionary[verse_reference] = verse_text

  return verse_dictionary

In [23]:
# Read the file and extract verse references
with open('/content/NVI2015.txt', 'r') as file:
    verses_2015 = file.readlines()

with open('/content/NVI2022.txt', 'r') as file:
    verses_2022 = file.readlines()

# Extract verse references using regular expressions
verse_2015_references = get_references(verses_2015)
verse_2022_references = get_references(verses_2022)

In [24]:
diff = []
for i in verse_2022_references:
  try:
    if verse_2022_references[i] != verse_2015_references[i]:
      diff.append(i)
  except KeyError:
    diff.append(i)

Get text with punctuation to use for the TTS model to improve pacing (originally would rush through lists without commas for example)

In [25]:
raw_verses = {}

for verse in verses_2022:
  match = re.match(r'^(\d*\s*[A-Za-z]+\s\d+:\d+(?:-\d+)?)\t(.*)', verse)
  if match:
      verse_reference = match.group(1)
      verse_text = match.group(2).strip()
      verse_text = verse_text.replace(':', ': ')

      if '-' in verse_reference:
          start_verse, end_verse = verse_reference.split('-')
          chapter, start_verse_number = start_verse.split(':')
          end_verse_number = end_verse
          for verse_number in range(int(start_verse_number), int(end_verse_number) + 1):
              if verse_number == int(start_verse_number):
                  raw_verses[f'{chapter}:{verse_number}'] = verse_text
              else:
                  raw_verses[f'{chapter}:{verse_number}'] = None
      else:
          raw_verses[verse_reference] = verse_text


Find out the proportion of different verses for each chapter

In [26]:
! wget 'https://huggingface.co/datasets/bible-nlp/biblenlp-corpus/raw/5c45ce386b8005844e7a11bcc8d12b99c69e984e/vref.txt'

--2023-06-12 15:17:08--  https://huggingface.co/datasets/bible-nlp/biblenlp-corpus/raw/5c45ce386b8005844e7a11bcc8d12b99c69e984e/vref.txt
Resolving huggingface.co (huggingface.co)... 18.67.0.55, 18.67.0.67, 18.67.0.90, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 431827 (422K) [text/plain]
Saving to: ‘vref.txt’

vref.txt            100%[===================>] 421.71K  --.-KB/s    in 0.07s   

2023-06-12 15:17:09 (5.70 MB/s) - ‘vref.txt’ saved [431827/431827]



In [27]:
verse_file = "/content/vref.txt"
verses_dict = {}

with open(verse_file, 'r') as file:
    for line in file:
        line = line.strip()
        chapter, verse = line.split(':')
        chapter = chapter.strip()
        
        if chapter in verses_dict:
            verses_dict[chapter] += 1
        else:
            verses_dict[chapter] = 1


In [28]:
verses_dict['GEN 10']

32

In [ ]:
diff_count = {}
diff_percent = {}

In [ ]:
for i in diff:
    chapter, verse = i.split(':')
    chapter = chapter.strip()
        
    if chapter in diff_count:
        diff_count[chapter] += 1
    else:
        diff_count[chapter] = 1

In [ ]:
for i in diff_count:
  try:
    num = int(diff_count[i])
    den = int(verses_dict[i])
    percentage = num/den
    diff_percent[i] = percentage
  except KeyError:
    pass

In [ ]:
diff_percent

{'GEN 1': 0.8387096774193549,
 'GEN 2': 0.56,
 'GEN 3': 0.5416666666666666,
 'GEN 4': 0.4230769230769231,
 'GEN 5': 0.59375,
 'GEN 6': 0.5454545454545454,
 'GEN 7': 0.2916666666666667,
 'GEN 8': 0.4090909090909091,
 'GEN 9': 0.5517241379310345,
 'GEN 10': 0.5,
 'GEN 11': 0.28125,
 'GEN 12': 0.55,
 'GEN 13': 0.6666666666666666,
 'GEN 14': 0.5,
 'GEN 15': 0.5238095238095238,
 'GEN 16': 0.5625,
 'GEN 17': 0.5185185185185185,
 'GEN 18': 0.696969696969697,
 'GEN 19': 0.3157894736842105,
 'GEN 20': 0.7222222222222222,
 'GEN 21': 0.6764705882352942,
 'GEN 22': 0.5833333333333334,
 'GEN 23': 0.6,
 'GEN 24': 0.40298507462686567,
 'GEN 25': 0.3235294117647059,
 'GEN 26': 0.5428571428571428,
 'GEN 27': 0.5869565217391305,
 'GEN 28': 0.3181818181818182,
 'GEN 29': 0.4857142857142857,
 'GEN 30': 0.5116279069767442,
 'GEN 31': 0.4074074074074074,
 'GEN 32': 0.48484848484848486,
 'GEN 33': 0.4,
 'GEN 34': 0.25806451612903225,
 'GEN 35': 0.5172413793103449,
 'GEN 36': 0.4418604651162791,
 'GEN 37': 0.

In [29]:
character_count = 0
for i in diff:
  character_count += len(raw_verses[i])

In [30]:
print(f'{len(diff)} verses are different.')
print(f'There are {character_count} total characters that need to be recorded.')

15249 verses are different.
There are 2056158 total characters that need to be recorded.


In [ ]:
import math
starter_char = [100000, 500000, 2000000]
start_price = [22, 99, 330]
cost_per_thou = [.30, .24, .18]
plans = ['Creator', 'Independent Publisher', 'Growing Business']

for i in range(len(starter_char)):
  remainder = character_count - starter_char[i]
  thousands = math.ceil(remainder/1000)
  extra_cost = thousands * cost_per_thou[i]
  total = start_price[i] + extra_cost
  print(f'The {plans[i]} plan would cost ${total}')

The Creator plan would cost $607.6
The Independent Publisher plan would cost $471.47999999999996
The Growing Business plan would cost $339.36


In [ ]:
for i in diff[:5]:
  print(verse_2015_references[i])
  print(verse_2022_references[i])

dios en el principio creó los cielos y la tierra
en el principio dios creó los cielos y la tierra
la tierra era un caos total las tinieblas cubrían el abismo y el espíritu de dios se movíasobre la superficie de las aguas
la tierra no tenía forma y estaba vacía las tinieblas cubrían el abismo y el espíritu de dios se movía sobre la superficie de las aguas
y dijo dios que exista la luz y la luz llegó a existir
y dijo dios que haya luz y la luz llegó a existir
dios consideró que la luz era buenay la separó de las tinieblas
dios consideró que la luz era buena y la separó de las tinieblas
a la luz la llamó día y a las tinieblas noche y vino la noche y llegó la mañana ese fue el primer día
a la luz la llamó día y a las tinieblas noche vino la noche y llegó la mañana ese fue el primer día


# Test the Cloned Voice

In [ ]:
custom_voice = voices[-2]
custom_voice.settings.stability = 1.0
custom_voice.settings.similarity_boost = 1.0

test_audio = generate(
  text= 'dios borró de la faz de la tierra a todo ser viviente desde los seres humanos hasta los ganados los reptiles y las aves del cielo Todos fueron borrados de la faz de la tierra solo quedaron noé y los que estaban con él en el arca',
  voice= custom_voice,
  model='eleven_multilingual_v1'
)

In [ ]:
play(test_audio, notebook = True)

# Generate New Audio Bible Portions

In [31]:
test_group = []

chapter = 'GEN 1'

for i in raw_verses:
  if f'{chapter}:' in i:
    test_group.append(i)

In [ ]:
test_group

['GEN 1:1',
 'GEN 1:2',
 'GEN 1:3',
 'GEN 1:4',
 'GEN 1:5',
 'GEN 1:6',
 'GEN 1:7',
 'GEN 1:8',
 'GEN 1:9',
 'GEN 1:10',
 'GEN 1:11',
 'GEN 1:12',
 'GEN 1:13',
 'GEN 1:14',
 'GEN 1:15',
 'GEN 1:16',
 'GEN 1:17',
 'GEN 1:18',
 'GEN 1:19',
 'GEN 1:20',
 'GEN 1:21',
 'GEN 1:22',
 'GEN 1:23',
 'GEN 1:24',
 'GEN 1:25',
 'GEN 1:26',
 'GEN 1:27',
 'GEN 1:28',
 'GEN 1:29',
 'GEN 1:30',
 'GEN 1:31']

In [32]:
len(test_group)

31

In [33]:
chapter_text = ''

for i in test_group:
  raw_verse = raw_verses[i].replace('«','"')
  raw_verse = raw_verse.replace('»','"')
  chapter_text += f'{raw_verse} '

chapter_text

'En el principio Dios creó los cielos y la tierra. La tierra no tenía forma y estaba vacía, las tinieblas cubrían el abismo y el Espíritu de Dios se movía sobre la superficie de las aguas. Y dijo Dios:  "¡Que haya luz!". Y la luz llegó a existir. Dios consideró que la luz era buena y la separó de las tinieblas. A la luz la llamó "día" y a las tinieblas, "noche". Vino la noche y llegó la mañana:  ese fue el primer día. Y dijo Dios:  "¡Que haya una expansión en medio de las aguas y que las separe!". Y así sucedió. Dios hizo la expansión que separó las aguas que están debajo de las aguas que están arriba. A esta expansión Dios la llamó "cielo". Vino la noche y llegó la mañana:  ese fue el segundo día. Y dijo Dios:  "¡Que las aguas debajo del cielo se reúnan en un solo lugar y que aparezca lo seco!". Y así sucedió. A lo seco Dios lo llamó "tierra" y al conjunto de aguas lo llamó "mares". Y Dios consideró que esto era bueno. Luego dijo Dios:  "¡Que haya vegetación sobre la tierra; que esta 

In [34]:
custom_voice = voices[-1]
custom_voice.settings.stability = 1.0
custom_voice.settings.similarity_boost = 1.0

verse_audio = generate(
    text= chapter_text,
    voice= custom_voice,
    model='eleven_multilingual_v1'
  )

In [35]:
save(verse_audio, f'/content/{chapter}.wav')

Download the folder as a .zip file.

In [ ]:
folder_path = f'/content/{chapter}'
zip_filename = f'{chapter}'

file = shutil.make_archive(zip_filename, 'zip', folder_path)

files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>